In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np


In [2]:

# Data augmentation and normalization for CIFAR-10
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

# Normalization for validation
valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

# Loading CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
valid_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=valid_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:

# Initializing a ResNet-18 model from scratch (no pre-trained weights)
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features

# Customizing the fully connected layer for the Plant Seedlings dataset
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, len(train_dataset.classes))
)

# Transfer the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:

# Setting up the criterion and different optimizers for experimentation
criterion = nn.CrossEntropyLoss()
optimizer_adam = optim.Adam(model.parameters(), lr=0.001)
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [5]:

def train_model(model, criterion, optimizer, num_epochs=10):
    model.train()
    train_loss, train_acc = [], []

    for epoch in range(num_epochs):
        running_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    return train_loss, train_acc

# Train the model with both optimizers for comparison
loss_adam, acc_adam = train_model(model, criterion, optimizer_adam)
loss_sgd, acc_sgd = train_model(model, criterion, optimizer_sgd)


Epoch 1/10, Loss: 1.6522, Accuracy: 0.4067
Epoch 2/10, Loss: 1.3241, Accuracy: 0.5421
Epoch 3/10, Loss: 1.1329, Accuracy: 0.6162
Epoch 4/10, Loss: 1.0144, Accuracy: 0.6584
Epoch 5/10, Loss: 0.9226, Accuracy: 0.6900
Epoch 6/10, Loss: 0.8488, Accuracy: 0.7168
Epoch 7/10, Loss: 0.7974, Accuracy: 0.7359
Epoch 8/10, Loss: 0.7514, Accuracy: 0.7485
Epoch 9/10, Loss: 0.7032, Accuracy: 0.7633
Epoch 10/10, Loss: 0.6693, Accuracy: 0.7751
Epoch 1/10, Loss: 0.5979, Accuracy: 0.7981
Epoch 2/10, Loss: 0.5461, Accuracy: 0.8161
Epoch 3/10, Loss: 0.5233, Accuracy: 0.8241
Epoch 4/10, Loss: 0.5050, Accuracy: 0.8293
Epoch 5/10, Loss: 0.4952, Accuracy: 0.8329
Epoch 6/10, Loss: 0.4834, Accuracy: 0.8366
Epoch 7/10, Loss: 0.4693, Accuracy: 0.8416
Epoch 8/10, Loss: 0.4605, Accuracy: 0.8448
Epoch 9/10, Loss: 0.4455, Accuracy: 0.8489
Epoch 10/10, Loss: 0.4400, Accuracy: 0.8507
